In [3]:
#export
from expDJ.nb_4Dj import *
from exp.nb_08 import *

In [2]:

# Get MNIST data

x_train,y_train,x_valid,y_valid = get_data()
type(x_train)

NameError: name 'datasets' is not defined

In [ ]:
#export
# Function to normalize the trainig data and validation data
def normalize_to(train, valid):
    m,s = train.mean(),train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [ ]:
# Normalize the x training and vaiation data
x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

In [ ]:
x_train.mean(), x_train.std()

In [ ]:
# Make the datasets
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid) 
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

# Make samplers

train_Smp, valid_Smp = SamplerDJ(train_ds, bs, shuffle=True), SamplerDJ(valid_ds, bs, shuffle=True)

# Make data loaders

train_dl, valid_dl = DataLoaderDJ(train_ds, train_Smp, bs), DataLoaderDJ(valid_ds, valid_Smp, bs)

# Make databunch
db = DataBunchDJ(train_dl, valid_dl, c = c)
db

In [ ]:
#export
# Reform MNIST data
class LambdaDJ(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x): return self.func(x)
    
def flatten(x):      return x.view(x.shape[0], -1)
def resize_MNIST(x): return x.view(-1, 1, 28, 28)
        
    


In [ ]:
x_train.shape,resize_MNIST(x_train).shape

In [ ]:
def get_cnn_model(data):
    return nn.Sequential(
        LambdaDJ(resize_MNIST),
        nn.Conv2d( 1, 8, 5, padding=2,stride=2), nn.ReLU(), #14
        nn.Conv2d( 8,16, 3, padding=1,stride=2), nn.ReLU(), # 7
        nn.Conv2d(16,32, 3, padding=1,stride=2), nn.ReLU(), # 4
        nn.Conv2d(32,32, 3, padding=1,stride=2), nn.ReLU(), # 2
        nn.AdaptiveAvgPool2d(1),
        LambdaDJ(flatten),
        nn.Linear(32,data.c)
    )

nn.Conv2d

In [ ]:
nn.Module
# Model
model = get_cnn_model(db)

In [ ]:
class earlyStop(Callback):
    _order = 1
    def __init__(self, stopIt = 3):
        self.stopIt = stopIt
        
    #def after_batch(self):
        #print(self.n_iter)
     #   if self.n_iter > 10:
       #     self.run.stop = True
      #      return True
        #print(self.stop) 
            
        #pass
        
    def after_epoch(self):
        print(self.run.epoch)
        if self.run.epoch == self.stopIt:
            self.run.early_stop.stopIt
            #self.stop = True
            return True

        
class printPreds(Callback):
    _order = 0
    def after_batch(self):
        print(run.pred[15])
        print(run.yb[15])
        
class accuracyCheckCb(Callback):
    _order = 1
    
    def __init__(self, m):
        self.m = m
        
    def after_batch(self):
        print(self.m(run.pred, run.yb))

# Callbacks
cbfs = [Recorder, partial(AvgStatsCallback,accuracy), partial(earlyStop, 3)]

In [ ]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = LearnerDJ(db, model, opt, loss_func)
run = RunnerDJ(cb_funcs=cbfs)



In [ ]:

%time run.fit(5, learn)
#run.iters

In [ ]:
# Cuda
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [ ]:
model = get_cnn_model(db)
opt = optim.SGD(model.parameters(), lr=0.4)
learn = LearnerDJ(db, model, opt, loss_func)
cbfs = [Recorder, partial(AvgStatsCallback,accuracy),CudaCallback]
run = RunnerDJ(cb_funcs=cbfs)

%time run.fit(5, learn)

In [ ]:
plt.imshow(x_valid[10].view(28,28))
pred = model.cpu()(x_valid[10:12]),y_valid,
x_valid[10].shape, 
#loss_func(pred[0], pred[1])
#loss_func(pred[0], pred[1][10:12])
#pred[1][10:11]
pred[0][1]

## Hooks

In [ ]:
act_means = []
act_stds = []

def append_stats(mod, inp, outp):
    act_means.append(outp.data.mean())
    act_stds.append(outp.data.std())

model = get_cnn_model(db)
opt = optim.SGD(model.parameters(), lr=0.4)
learn = LearnerDJ(db, model, opt, loss_func)
cbfs = [Recorder, partial(AvgStatsCallback,accuracy),CudaCallback]
run = RunnerDJ(cb_funcs=cbfs)

model[3].register_forward_hook(append_stats)
%time run.fit(5, learn)   



In [ ]:
plt.plot(act_means),plt.plot(act_stds)

## Hooks class

In [ ]:
#export
class ListContainer():
    def __init__(self, items): self.items = listify(items)
    def __getitem__(self, idx):
        if isinstance(idx, (int,slice)): return self.items[idx]
        if isinstance(idx[0],bool):
            assert len(idx)==len(self) # bool mask
            return [o for m,o in zip(idx,self.items) if m]
        return [self.items[i] for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, o): self.items[i] = o
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res
    
class Hook():
    def __init__(self, mLyr, func):
        self.hook = mLyr.register_forward_hook(partial(func, self))
        
    def remove(self): self.hook.remove()
    def __del__(self): self.remove()

def append_stats(hook, mod, inp, outp):
    if not hasattr(hook,'stats'): hook.stats = ([],[], [])
    means,stds, meds = hook.stats
    means.append(outp.data.mean())
    stds.append(outp.data.std())
    meds.append(outp.data.median())
    
## test 

model = get_cnn_model(db)
opt = optim.SGD(model.parameters(), lr=0.4)
learn = LearnerDJ(db, model, opt, loss_func)
cbfs = [Recorder, partial(AvgStatsCallback,accuracy),CudaCallback]


hook = Hook(model[1], append_stats)

run = RunnerDJ(cb_funcs=cbfs)
run.fit(1, learn)
        
    
from torch.nn import init

class HooksDJ(ListContainer):
    def  __init__(self, f, ms): # f is the function, m is the model
        super().__init__([Hook(m, f) for m in ms])
    
    
    def __enter__(self, *args): return self
        
        
    def __exit__(self, *args): self.remove()
    
    def __del__(self):
        self.remove()
    
    def __delitem__(self, i):
        self[i].remove
        super.__delitem__(i)
    
    
    def remove(self):
        
        for dx in self:
            dx.remove()
    





In [ ]:

plt.plot(hook.stats[1])
plt.plot(hook.stats[0])
plt.plot(hook.stats[2])
hook.remove()

## Run with Hooks container

In [ ]:
def append_stats_hist(hook, mod, inp, outp):
    if not hasattr(hook,'stats'): hook.stats = ([],[], [])
    means,stds, hists = hook.stats
    means.append(outp.data.mean().cpu())
    stds.append(outp.data.std().cpu())
    #meds.append(outp.data.median())
    hists.append(outp.data.cpu().histc(40,0,10)) #histc isn't implemented on the GPU

model = get_cnn_model(db)
opt = optim.SGD(model.parameters(), lr=0.9)
learn = LearnerDJ(db, model, opt, loss_func)
cbfs = [Recorder, partial(AvgStatsCallback,accuracy),CudaCallback]

with HooksDJ(append_stats_hist, model) as hooks: run.fit(1, learn)


In [ ]:
# Thanks to @ste for initial version of histgram plotting code
def get_hist(h): return torch.stack(h.stats[2]).t().float().log1p()

fig,axes = plt.subplots(3,2, figsize=(15,6))
for ax,h in zip(axes.flatten(), hooks[:6]):
    ax.imshow(get_hist(h), origin='lower')
    ax.axis('off')#
plt.tight_layout()

In [ ]:
# hooks[1] this is the layer 
len(hooks[0].stats[0])

In [ ]:
opt

In [ ]:
len(train_ds)

In [ ]:
512*40

In [ ]:
512*118

In [ ]:
model = get_cnn_model(db)
modelOrig = model
dx = 5
print(modelOrig[dx].weight.std())

class GeneralRelu(nn.Module):
    def __init__(self, leak=None, sub=None, maxv=None):
        super().__init__()
        self.leak,self.sub,self.maxv = leak,sub,maxv

    def forward(self, x): 
        x = F.leaky_relu(x,self.leak) if self.leak is not None else F.relu(x)
        if self.sub is not None: x.sub_(self.sub)
        if self.maxv is not None: x.clamp_max_(self.maxv)
        return x

def init_cnn(m, uniform=False):
    f = init.kaiming_uniform_ if uniform else init.kaiming_normal_
    for l in m:
        if isinstance(l, nn.Conv2d):
            #print("Boom")
            print("Pre: ")
            print(l.weight.std())
            f(l.weight, a=0)
            print("post: ")
            print(l.weight.std())
            l.bias.data.zero_()

        
init_cnn(model)

rel = GeneralRelu(leak = 0.1)
modelOrig[dx].weight.mean(), modelOrig[dx].weight.std()
#model[0]
#F.leaky_relu_(model[1].weight)
F.leaky_relu(model[1].weight[:], 0).std(), model[1].weight.std()



## Hooks and stuff

